In [1]:
import nibabel as nb
import matplotlib.pyplot as plt
import glob, os
import numpy as np

In [1]:
folder_path = "/vol/ml/pding/quickNAT_pytorch/datasets/OASIS/"

In [27]:
print(unique_mal)
print(unique_nor)

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32.]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32.]


## get the freesurfer label and numerical mapping

In [5]:
with open("/vol/ml/ngetty/oasis/fs_idx2label.csv", 'r') as f:
    raw_pairs = [each.split(',') for each in f.read().splitlines()]
    fs_idx_label = {int(each[0]) : each[1] for each in raw_pairs }

with open("/vol/ml/ngetty/oasis/anat_vocab_full.txt", 'r') as f:
    raw_pairs = [each.split() for each in f.read().splitlines()]
    fs_idx_idx = {int(each[1]) : int(each[0]) for each in raw_pairs}

idx_label = {}
idx_label[0] = 'background'
for key, value in fs_idx_idx.items():
    idx_label[key] = fs_idx_label[value]

In [12]:
list(idx_label.values())

['background',
 'lh_thalamus',
 'lh_caudate',
 'lh_putamen',
 'lh_pallidum',
 'lh_hippocampus',
 'lh_amygdala',
 'lh_acumbens',
 'rh_thalamus',
 'rh_caudate',
 'rh_putamen',
 'rh_pallidum',
 'rh_hippocampus',
 'rh_amygdala',
 'rh_acumbens',
 'lh.bankssts',
 'lh.caudalanteriorcingulate',
 'lh.caudalmiddlefrontal',
 'lh.cuneus',
 'lh.entorhinal',
 'lh.fusiform',
 'lh.inferiorparietal',
 'lh.inferiortemporal',
 'lh.isthmuscingulate',
 'lh.lateraloccipital',
 'lh.lateralorbitofrontal',
 'lh.lingual',
 'lh.medialorbitofrontal',
 'lh.middletemporal',
 'lh.parahippocampal',
 'lh.paracentral',
 'lh.parsopercularis',
 'lh.parsorbitalis',
 'lh.parstriangularis',
 'lh.pericalcarine',
 'lh.postcentral',
 'lh.posteriorcingulate',
 'lh.precentral',
 'lh.precuneus',
 'lh.rostralanteriorcingulate',
 'lh.rostralmiddlefrontal',
 'lh.superiorfrontal',
 'lh.superiorparietal',
 'lh.superiortemporal',
 'lh.supramarginal',
 'lh.frontalpole',
 'lh.temporalpole',
 'lh.transversetemporal',
 'lh.insula',
 'rh.ba

# Pre-train

In [13]:
from settings import Settings
settings = Settings()

common_params, data_params, net_params, train_params, eval_params = settings['COMMON'], settings['DATA'], settings[
        'NETWORK'], settings['TRAINING'], settings['EVAL']

In [14]:
common_params

{'save_model_dir': 'saved_models',
 'model_name': 'quicknat',
 'log_dir': 'logs',
 'device': 0,
 'exp_dir': 'experiments'}

In [15]:
data_params, net_params

({'data_dir': 'datasets/OASIS/axial',
  'train_data_file': 'Data_train.h5',
  'train_label_file': 'Label_train.h5',
  'train_class_weights_file': 'Class_Weight_train.h5',
  'train_weights_file': 'Weight_train.h5',
  'test_data_file': 'Data_test.h5',
  'test_label_file': 'Label_test.h5',
  'test_class_weights_file': 'Class_Weight_test.h5',
  'test_weights_file': 'Weight_test.h5',
  'labels': ['background',
   'lh_thalamus',
   'lh_caudate',
   'lh_putamen',
   'lh_pallidum',
   'lh_hippocampus',
   'lh_amygdala',
   'lh_acumbens',
   'rh_thalamus',
   'rh_caudate',
   'rh_putamen',
   'rh_pallidum',
   'rh_hippocampus',
   'rh_amygdala',
   'rh_acumbens',
   'lh.bankssts',
   'lh.caudalanteriorcingulate',
   'lh.caudalmiddlefrontal',
   'lh.cuneus',
   'lh.entorhinal',
   'lh.fusiform',
   'lh.inferiorparietal',
   'lh.inferiortemporal',
   'lh.isthmuscingulate',
   'lh.lateraloccipital',
   'lh.lateralorbitofrontal',
   'lh.lingual',
   'lh.medialorbitofrontal',
   'lh.middletemporal',

In [ ]:
from run import train

train(train_params, common_params, data_params, net_params)

In [ ]:
## Convert OASIS Data Path

In [25]:
def load_path_img_label(path_to_dataset):
    '''
    This function receives the "path_to_dataset" which should contain both the 
    NIFTI format data for both brain images and corresponding labels
    return the collections of path_to_file, a list with each entry contains: 
        [0] path to *.nii.gz brain image
        [1] path to *.nii.gz brain label
    '''
    path_to_imgs = os.path.join(path_to_dataset, "*_brain.nii.gz")
    path_to_labels = os.path.join(path_to_dataset, "*_labels.nii.gz")
    labels = glob.glob(path_to_labels)
    images = glob.glob(path_to_imgs)
    labels.sort()
    images.sort()
    return tuple(zip(images, labels))

In [26]:
paths_to_imgs_labels = load_path_img_label(folder_path)
print(paths_to_imgs_labels[:4])

total_len = len(paths_to_imgs_labels)

[('/vol/ml/ngetty/oasis/segmented/OAS30010_MR_d0068_brain.nii.gz', '/vol/ml/ngetty/oasis/segmented/OAS30010_MR_d0068_labels.nii.gz'), ('/vol/ml/ngetty/oasis/segmented/OAS30011_MR_d0055_brain.nii.gz', '/vol/ml/ngetty/oasis/segmented/OAS30011_MR_d0055_labels.nii.gz'), ('/vol/ml/ngetty/oasis/segmented/OAS30011_MR_d1671_brain.nii.gz', '/vol/ml/ngetty/oasis/segmented/OAS30011_MR_d1671_labels.nii.gz'), ('/vol/ml/ngetty/oasis/segmented/OAS30012_MR_d0124_brain.nii.gz', '/vol/ml/ngetty/oasis/segmented/OAS30012_MR_d0124_labels.nii.gz')]


## Divide into training and validation set

In [24]:
train_ratio = .98
valid_ratio = 1 - train_ratio

train_len = int(train_ratio * total_len)
valid_len = total_len - train_len

train_idx = np.random.choice(total_len, train_len, replace=False)
valid_idx = np.array([i for i in range(total_len) if i not in train_idx])

train_file_paths = [paths_to_imgs_labels[i] for i in train_idx]
valid_file_paths = [paths_to_imgs_labels[i] for i in valid_idx]

## preprocess the nifti data into different perspective and store into h5 file

In [4]:
!pwd

/vol/ml/pding/quickNAT_pytorch


In [3]:
# Sagittal view
!python utils/convert_h5.py -dd /vol/ml/ngetty/oasis/segmented/ -ld /vol/ml/ngetty/oasis/segmented/ -ds 98,2 -rc FS -o SAG -df datasets/OASIS/sagittal/

* Start *
Total no of volumes to process : 1585
Train dataset size: 1553, Test dataset size: 32
===Train data===
Loading and preprocessing data...
#########^C
Traceback (most recent call last):
  File "utils/convert_h5.py", line 119, in <module>
    args.orientation)
  File "utils/convert_h5.py", line 64, in convert_h5
    remove_black=True)
  File "/vol/ml/pding/quickNAT_pytorch/utils/data_utils.py", line 87, in load_dataset
    return_weights=return_weights)
  File "/vol/ml/pding/quickNAT_pytorch/utils/data_utils.py", line 120, in load_and_preprocess
    return_weights=return_weights)
  File "/vol/ml/pding/quickNAT_pytorch/utils/data_utils.py", line 140, in preprocess
    volume, labelmap = preprocessor.reduce_slices(volume, labelmap)
  File "/vol/ml/pding/quickNAT_pytorch/utils/preprocessor.py", line 74, in reduce_slices
    data_reduced = np.compress(mask_vector, data, axis=0).reshape(-1, H, W)
  File "<__array_function__ internals>", line 6, in compress
  File "/vol/ml/pding/minic

In [ ]:
# Coronal view
!python utils/convert_h5.py -dd /vol/ml/ngetty/oasis/segmented/ -ld /vol/ml/ngetty/oasis/segmented/ -ds 98,2 -rc FS -o COR -df datasets/OASIS/coronal/

* Start *
Total no of volumes to process : 1585
Train dataset size: 1553, Test dataset size: 32
===Train data===
Loading and preprocessing data...


In [5]:
# Axial view
!python utils/convert_h5.py -dd /vol/ml/ngetty/oasis/segmented/ -ld /vol/ml/ngetty/oasis/segmented/ -ds 98,2 -rc FS -o AXI -df datasets/OASIS/axial/

^C
Traceback (most recent call last):
  File "utils/convert_h5.py", line 15, in <module>
    import data_utils as du
  File "/vol/ml/pding/quickNAT_pytorch/utils/data_utils.py", line 4, in <module>
    import nibabel as nb
  File "/vol/ml/pding/miniconda3/lib/python3.6/site-packages/nibabel/__init__.py", line 93, in <module>
    from . import streamlines
  File "/vol/ml/pding/miniconda3/lib/python3.6/site-packages/nibabel/streamlines/__init__.py", line 12, in <module>
    from .trk import TrkFile
  File "<frozen importlib._bootstrap>", line 971, in _find_and_load
  File "<frozen importlib._bootstrap>", line 955, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 665, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 674, in exec_module
  File "<frozen importlib._bootstrap_external>", line 779, in get_code
  File "<frozen importlib._bootstrap_external>", line 487, in _compile_bytecode
KeyboardInterrupt


## Train the model

In [2]:
!python run.py --mode=train

Traceback (most recent call last):
  File "run.py", line 7, in <module>
    from quicknat import QuickNat
  File "/vol/ml/pding/quickNAT_pytorch/quicknat.py", line 5, in <module>
    from nn_common_modules import modules as sm
ModuleNotFoundError: No module named 'nn_common_modules'
